# Plant Pathology Hypertuning

### Setup

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
import copy

from utils import utils, learner
from utils import PlantPathologyDataset as dataset

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

/home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [5]:
print_every = 100
assert(torch.cuda.is_available())
gpu_dtype = torch.cuda.FloatTensor

In [6]:
NUM_TRAIN = 75
NUM_VAL = 25
thomas = False

if thomas:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
else:
    csv_file='./train.csv'
    root_dir='./train_images'
    mappings_dir='./labelMappings.csv'

plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir)
train_data = DataLoader(plant_dataset, batch_size=10, sampler= learner.ChunkSampler(NUM_TRAIN, 0))
validation_data = DataLoader(plant_dataset, batch_size=10, sampler=learner.ChunkSampler(NUM_VAL, NUM_TRAIN))

### Testing resnet50

In [7]:
resnet50 = models.resnet50()#.cuda() # pretrained=True is pretrained on ImageNet

loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)

In [9]:
learner.train(resnet50, train_data, loss_fn, optimizer, print_every=5)
learner.check_accuracy(resnet50, validation_data)

Starting epoch 1 / 1
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
t = 5, loss = 14.6753
torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
Checking accuracy on validation set
Got 1 / 25 correct (4.00)


0.04

In [10]:
### Hypertuning